In [10]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [11]:
import pandas as pd
import numpy as np
data=pd.read_csv("mfcc_value.csv")
print(data.shape)
y_data=data['code']
print(y_data.shape)
x_data=data.iloc[:,0:128]
print(x_data.shape)

(1437, 130)
(1437,)
(1437, 128)


In [12]:
from  sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.20,random_state=42)
print(type(x_train))
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(len(x_train))


print(type(x_train))

<class 'pandas.core.frame.DataFrame'>
(288, 128)
(1149,)
(288,)
1149
<class 'numpy.ndarray'>


In [13]:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


no_rows=len(x_train)
training_epochs = 350
learning_rate = 0.001
batch_size = 40
display_step = 10
n_classes=16
n_input= x_train.shape[1]
print(n_input)
n_hidden_1= 128
n_hidden_2= 128
n_hidden_3= 128
n_hidden_4=128
n_hidden_5=128





128


In [14]:
y_train = tf.one_hot(y_train, n_classes)
y_test = tf.one_hot(y_test, n_classes)

#Build the model

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [15]:
## First Layer

h1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
b1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.nn.selu(x @ h1 + b1)

## second layer
h2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
b2 = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2 = tf.nn.selu(layer_1 @ h2 + b2)

## third layer
h3 = tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]))
b3 = tf.Variable(tf.random_normal([n_hidden_3]))
layer_3 =tf.nn.selu(layer_2 @ h3 + b3)

## fourth layer
h4 = tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4]))
b4 = tf.Variable(tf.random_normal([n_hidden_4]))
layer_4 =tf.nn.selu(layer_2 @ h4 + b4)

## fifth layer
h5 = tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5]))
b5 = tf.Variable(tf.random_normal([n_hidden_5]))
layer_5 = tf.nn.selu(layer_2 @ h5 + b5)


##Final Output layer


w = tf.Variable(tf.random_normal([n_hidden_5, n_classes]))
b_out= tf.Variable(tf.random_normal([n_classes]))
out_layer = layer_5 @ w + b_out

In [16]:
##cost
cost = tf.reduce_mean(tf.nn.\
                      softmax_cross_entropy_with_logits_v2(\
        logits=out_layer, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=\
                                   learning_rate).minimize(cost)
seed = 30
tf.set_random_seed(seed)
a = tf.random_uniform([batch_size],\
                      minval=0,maxval=len(x_train),\
                      dtype=tf.int64)
print(a)

init = tf.global_variables_initializer()
total_batch = int(len(x_train)/batch_size)

Tensor("random_uniform_1:0", shape=(40,), dtype=int64)


In [17]:
with tf.Session() as sess:
    sess.run(init)
    y_train=y_train.eval()
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train)/batch_size)
        for i in range(batch_size):
            #print('i', i)
            a1=sess.run(a)
            batch_xs = x_train[a1,:]
            batch_ys = y_train[a1]
                     
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += sess.run(cost,feed_dict= {x: batch_xs,y: batch_ys})/total_batch
            #print(avg_cost)
        if epoch % display_step == 0: 
            print ("Epoch:", '%04d' % (epoch+1),\
                    "cost=", "{:.9f}".format(avg_cost))
    print ("Training phase finished")
    # Test model
    correct_prediction = tf.equal (tf.argmax(out_layer, 1),tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Model accuracy:", accuracy.eval({x: x_test,y:y_test.eval()}))
    
    

Epoch: 0001 cost= 244673.022042411
Epoch: 0011 cost= 11994.410278320
Epoch: 0021 cost= 7073.056335449
Epoch: 0031 cost= 4930.208182199
Epoch: 0041 cost= 3890.512939453
Epoch: 0051 cost= 3031.843898228
Epoch: 0061 cost= 2464.628104074
Epoch: 0071 cost= 2241.383268084
Epoch: 0081 cost= 1895.681276594
Epoch: 0091 cost= 1935.439176287
Epoch: 0101 cost= 1746.607045855
Epoch: 0111 cost= 1630.820144653
Epoch: 0121 cost= 1322.501944406
Epoch: 0131 cost= 1142.740009308
Epoch: 0141 cost= 843.727005005
Epoch: 0151 cost= 1509.287802560
Epoch: 0161 cost= 1235.167556218
Epoch: 0171 cost= 1064.389302662
Epoch: 0181 cost= 811.200398036
Epoch: 0191 cost= 694.455720629
Epoch: 0201 cost= 702.104039873
Epoch: 0211 cost= 791.595409802
Epoch: 0221 cost= 741.597745078
Epoch: 0231 cost= 1093.330514635
Epoch: 0241 cost= 518.533232689
Epoch: 0251 cost= 652.946248736
Epoch: 0261 cost= 858.820092882
Epoch: 0271 cost= 476.968340465
Epoch: 0281 cost= 1292.790656771
Epoch: 0291 cost= 479.580578600
Epoch: 0301 cost= 